In [1]:
import pandas as pd

from ppgmne_prf.load_data import load_data
from ppgmne_prf.preprocess import preprocess_accidents, preprocess, preprocess_stations
from ppgmne_prf.config.paths import PATH_DATA_IBGE_UF
from ppgmne_prf.config.params import N_CLUSTERS, CLUSTER_DMAX

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
df_accidents, dict_stations = load_data()

2023-08-08 18:32:49.812 | INFO     | ppgmne_prf.load_data:load_data:16 - Load data - Início do carregamento os dados de entrada.
2023-08-08 18:32:49.832 | INFO     | ppgmne_prf.load_data:load_data:18 - Load data (accidents) - Carregando os dados históricos dos acidentes.
2023-08-08 18:32:49.837 | WARNING  | ppgmne_prf.load_data:__load_accidents:44 - Load data (accidents) - Dados carregados da cache.
2023-08-08 18:32:50.016 | INFO     | ppgmne_prf.load_data:load_data:21 - Load data (stations) - Carregando as coordenadas das UOPs e delegacias.
2023-08-08 18:32:50.017 | WARNING  | ppgmne_prf.load_data:__load_stations:132 - Load data (stations) - Dados carregados da cache.
2023-08-08 18:32:50.017 | INFO     | ppgmne_prf.load_data:load_data:26 - Load data - Fim do carregamento os dados de entrada.


In [6]:
df_accidents[df_accidents['ano'] == 2022].shape

(64547, 28)

In [5]:
df_accidents.head()

,id,data_inversa,dia_semana,horario,uf,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano
0,100027,2018-01-01,segunda-feira,06:00:00,TO,RIO DOS BOIS,Falta de Atenção à Condução,Colisão frontal,Com Vítimas Feridas,Pleno dia,Decrescente,Simples,Não Informado,Não,2,0,0,1,1,0,1,2,"-9,35641407","-48,54610294",SPRF-TO,DEL02-TO,UOP02-DEL02-TO,2018
1,100044,2018-01-01,segunda-feira,08:30:00,SE,ITAPORANGA DAJUDA,Não guardar distância de segurança,Colisão traseira,Sem Vítimas,Pleno dia,Decrescente,Dupla,Reta,Não,2,0,0,0,2,0,0,2,"-11,04476007","-37,36498",SPRF-SE,DEL02-SE,UOP01-DEL02-SE,2018
2,100046,2018-01-01,segunda-feira,08:50:00,TO,GUARAI,Falta de Atenção à Condução,Colisão lateral,Sem Vítimas,Pleno dia,Crescente,Simples,Não Informado,Não,2,0,0,0,2,0,0,2,"-8,952295","-48,500107",SPRF-TO,DEL02-TO,UOP02-DEL02-TO,2018
3,100052,2018-01-01,segunda-feira,05:30:00,PA,ELDORADO DO CARAJAS,Ingestão de Álcool,Colisão lateral,Com Vítimas Fatais,Amanhecer,Crescente,Simples,Reta,Não,2,1,0,0,1,0,0,2,"-6,345512","-49,389626",SPRF-PA,DEL03-PA,UOP01-DEL03-PA,2018
4,100053,2018-01-01,segunda-feira,10:25:00,BA,TUCANO,Ultrapassagem Indevida,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,Crescente,Simples,Reta,Não,1,0,0,1,0,0,1,1,"-11,08661652","-38,83420181",SPRF-BA,DEL02-BA,DEL10/2-UOP02/BA,2018


In [13]:
df_2022 = df_accidents[df_accidents['ano'] == 2022].groupby(['regional', 'ano']).agg(n=('id', 'count')).reset_index()
df_2022.sort_values(by='n', ascending=False)

,regional,ano,n
10,SPRF-MG,2022,8274
23,SPRF-SC,2022,7621
17,SPRF-PR,2022,7335
22,SPRF-RS,2022,4940
18,SPRF-RJ,2022,4704
25,SPRF-SP,2022,4376
4,SPRF-BA,2022,3354
15,SPRF-PE,2022,2817
7,SPRF-ES,2022,2278
8,SPRF-GO,2022,2204


In [14]:
df_2021 = df_accidents[df_accidents['ano'] == 2021].groupby(['regional', 'ano']).agg(n=('id', 'count')).reset_index()
df_2021.sort_values(by='n', ascending=False)

,regional,ano,n
10,SPRF-MG,2021,8305
23,SPRF-SC,2021,7918
17,SPRF-PR,2021,7307
18,SPRF-RJ,2021,4565
22,SPRF-RS,2021,4545
25,SPRF-SP,2021,4293
4,SPRF-BA,2021,3230
15,SPRF-PE,2021,2627
7,SPRF-ES,2021,2525
8,SPRF-GO,2021,2193


In [15]:
df_2020 = df_accidents[df_accidents['ano'] == 2020].groupby(['regional', 'ano']).agg(n=('id', 'count')).reset_index()
df_2020.sort_values(by='n', ascending=False)

,regional,ano,n
10,SPRF-MG,2020,8356
23,SPRF-SC,2020,7247
17,SPRF-PR,2020,7162
18,SPRF-RJ,2020,4238
22,SPRF-RS,2020,4188
25,SPRF-SP,2020,4032
4,SPRF-BA,2020,3361
15,SPRF-PE,2020,2643
7,SPRF-ES,2020,2534
12,SPRF-MT,2020,2182


In [16]:
df_2019 = df_accidents[df_accidents['ano'] == 2019].groupby(['regional', 'ano']).agg(n=('id', 'count')).reset_index()
df_2019.sort_values(by='n', ascending=False)

,regional,ano,n
10,SPRF-MG,2019,8722
23,SPRF-SC,2019,8476
17,SPRF-PR,2019,7675
18,SPRF-RJ,2019,4614
22,SPRF-RS,2019,4603
25,SPRF-SP,2019,4372
4,SPRF-BA,2019,3422
7,SPRF-ES,2019,2846
15,SPRF-PE,2019,2798
8,SPRF-GO,2019,2275


In [17]:
df_2018 = df_accidents[df_accidents['ano'] == 2018].groupby(['regional', 'ano']).agg(n=('id', 'count')).reset_index()
df_2018.sort_values(by='n', ascending=False)

,regional,ano,n
10,SPRF-MG,2018,9050
23,SPRF-SC,2018,8530
17,SPRF-PR,2018,7920
18,SPRF-RJ,2018,4577
25,SPRF-SP,2018,4511
22,SPRF-RS,2018,4456
4,SPRF-BA,2018,3517
15,SPRF-PE,2018,2770
7,SPRF-ES,2018,2642
12,SPRF-MT,2018,2623
